In [3]:
import numpy as np
import json
import pandas as pd
import os
import re
import ast
import xml.etree.ElementTree as ET
import ast

from __future__ import print_function
from gensim import corpora

from nltk.sem.logic import *
from nltk.sem.logic import LogicParser

In [9]:
#ここを外部読み込みにさせたい.
variable_arr = []
pre_var = []
formulas = []
check_dup = set([])

from nltk.sem.logic import LogicParser
from nltk.sem.logic import *

logic_parser = LogicParser(type_check=False)
def lexpr(formula_str):
    return logic_parser.parse(formula_str)

def coq_string_expr(expression):
    org_exp = expression
    
    if isinstance(expression, str):
        expression = lexpr(expression)
    expr_coq_str = ''
    if isinstance(expression, ApplicationExpression):
        coq_string_application_expr(expression)
    elif isinstance(expression, AbstractVariableExpression):
        expr_coq_str = coq_string_abstract_variable_expr(expression)
    elif isinstance(expression, LambdaExpression):
        #print('lambda',expression)
        expr_coq_str = str(expression)
        #expr_coq_str = 'lambda error'
    elif isinstance(expression, QuantifiedExpression):
        expr_coq_str = coq_string_quantified_expr(expression)
    elif isinstance(expression, AndExpression):
        expr_coq_str = coq_string_and_expr(expression)
    elif isinstance(expression, OrExpression):
        expr_coq_str = coq_string_or_expr(expression)
    elif isinstance(expression, NegatedExpression):
        expr_coq_str = coq_string_not_expr(expression)
    elif isinstance(expression, BinaryExpression):
        expr_coq_str = coq_string_binary_expr(expression)
    elif isinstance(expression, Variable):
        expr_coq_str = '%s' % expression
    else:
        expr_coq_str = str(expression)
    return org_exp

coqstr = coq_string_expr

def coq_string_application_expr(expression):
    # uncurry the arguments and find the base function
    if expression.is_atom():
        #is_atom : 原子論理式かどうか
        function, args = expression.uncurry()
        arg_str = ' '.join("%s" % coqstr(arg) for arg in args)
    else:
        #Leave arguments curried
        function = expression.function
        arg_str = "%s" % coqstr(expression.argument)
    function_str = "%s" % coqstr(function)
    parenthesize_function = False
    if isinstance(function, LambdaExpression):
        if isinstance(function.term, ApplicationExpression):
            if not isinstance(function.term.function,
                              AbstractVariableExpression):
                parenthesize_function = True
        elif not isinstance(function.term, BooleanExpression):
            parenthesize_function = True
    elif isinstance(function, ApplicationExpression):
        parenthesize_function = True
    if parenthesize_function:
        function_str = Tokens.OPEN + function_str + Tokens.CLOSE
    return expression #Tokens.OPEN + function_str + ' ' + arg_str + Tokens.CLOSE


def coq_string_abstract_variable_expr(expression):
    expr_str = str(expression.variable)
    if not isinstance(expression, FunctionVariableExpression):
        if expr_str == '':
            expr_str = "%s" % expr_str
        else:
            expr_str = "%s" % expr_str
    else:
        expr_str = "%s" % expr_str
        
    if not re.sub(r'_', "",expr_str) in check_dup:
        if not expr_str.startswith('_'):
            pre_var.append(expr_str)
            check_dup.add(expr_str)
    return expression


def coq_string_quantified_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    for v in variables:
        coqstr(v)
    coqstr(term) 
    return expression

def coq_string_and_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression 

def coq_string_or_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression

def coq_string_not_expr(expression):
    term_str = coqstr(expression.term)
    return expression

def coq_string_binary_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression


In [10]:
def substituteString(text,lst):
    ###辞書作り###
    lst = list(set(lst))
    e_sub = {}
    z_sub = {}
    ice = 0 #index counter
    icz = 0 

    for i in lst:
        matchObj = re.search(r'e0+', i)
        if(matchObj):
            tmp = "e0"+str(ice)
            e_sub[tmp] = i
            ice+=1
            continue
        matchObj = re.search(r'z[0-9]*', i)
        if matchObj:
            tmp = "z0"+str(icz)
            z_sub[tmp] = i
            icz+=1
    
    e_sub = {v:k for k, v in e_sub.items()}
    z_sub = {v:k for k, v in z_sub.items()}
    
    for k, v in e_sub.items():  
        text = text.replace(k,v)
    for k, v in z_sub.items():  
        text = text.replace(k,v)
    return text

In [11]:
#en_parsed のsemから取り出し ただし，論理記号の順番がスタック式をなおさなければならなそう
tree = ET.parse('sem/snli_dev.txt.candc.sem.xml')
root = tree.getroot()

root = root[0]
root = root[0] #１個目のsentence

mydict = {}
c = 0 #辞書のindexを回す
print(len(root))

#子階層のタグと中身
for child in root:
    
    formula = child[2]  #child[2]がsemantics
    check = (child[2].attrib)
    check = check['status']
   
    if(check == 'success'):
        plain = ""
        toridashi = child[0]
        
        for i in toridashi :
            p = i.attrib
            p = p['surf']
            if(p=='.' or p== ','):
                plain = plain + p
            else :
                plain = plain + " " + p
        plain = plain+'\n'
        
        #print(plain)
        
        formula = child[2][0]
        formula = (formula.attrib)
        formula = formula['sem'] #\nが付与されていると信じている
        
        #論理式を今回の標準にする
        check_dup = set([])
        pre_var = []
        
        try:
            formula = coq_string_expr(formula)
            formula = substituteString(formula,pre_var)
            ice = 0
            icz = 0
            
            for i in range(len(pre_var)):
                matchObj = re.search(r'e0+', pre_var[i])
                if(matchObj):
                    tmp = "e0"+str(ice)
                    pre_var[i] = tmp
                    
                matchObj = re.search(r'z[0-9]*', pre_var[i])
                if matchObj:
                    tmp = "z0"+str(icz)
                    pre_var[i] = tmp
                    icz+=1
                    
            variable_arr.append(pre_var)
            pre_var = []
            pair = {'text':plain,'formula':formula}
            mydict.update({str(c):pair})
        except:
            import traceback
            traceback.print_exc()
            pre_var = []
    else:
      print('faild parse')
    c+=1
    #if(c==10):
    #    break
#print(mydict)
    
f = open('snli_input_data_1213.json', 'w') # 書き込みモードで開く
json.dump(mydict, f,ensure_ascii=False)
f.close()

19949


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 290, in handle
    return self.handle_lambda(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 363, in handle_lambda
    accum = self.process_next_expression(tok)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 296, in handle
    return self.handle_open(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 406, in handle_open
    accum = self.process_next_expression(None)
  File "

faild parse
faild parse

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 278, in process_next_expression
    return self.attempt_adjuncts(accum, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 303, in attempt_adjuncts
    expression = self.attempt_ApplicationExpression(expression, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 473, in attempt_ApplicationExpression
    "' is not a Lambda Expression, an "
nltk.sem.logic.LogicalExpressionException: The function '(_long(\x.TrueP) & \K.TrueP(\x.exists z8161.(_sweep(z8161) & TrueP & exists e.(_boot(e) & (Subj(e) = x) & (Acc(e) = z8161) & _outside(e) & K(e)))))' is not a Lambda Expression, an Application Expression, or a functional predicate, so it may not take arguments.

During handling of the above exception,


faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse


...

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 290, in handle
    return self.handle_lambda(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 363, in handle_lambda
    accum = self.process_next_expression(tok)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 296, in handle
    return self.handle_open(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 406, in handle_open
    accum = self.process_next_expression(None)
  File "

faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse
faild parse


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 290, in handle
    return self.handle_lambda(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 363, in handle_lambda
    accum = self.process_next_expression(tok)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 296, in handle
    return self.handle_open(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 406, in handle_open
    accum = self.process_next_expression(None)
  File "

faild parse
faild parse
faild parse


Traceback (most recent call last):
  File "<ipython-input-11-55c1fef81fcf>", line 43, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-9-425e32d3ff24>", line 18, in coq_string_expr
    expression = lexpr(expression)
  File "<ipython-input-9-425e32d3ff24>", line 12, in lexpr
    return logic_parser.parse(formula_str)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 147, in parse
    raise LogicalExpressionException(None, msg)
nltk.sem.logic.LogicalExpressionException: The function '(_scene(x) & X0(x) & exists z49635.(_show(z49635) & _life(z49635) & exists z49630.(_sing(z49630) & _man(z49630) & exists z49628.(_woman(z49628) & _one(z49628) & _two(z49628) & all z49627.(_side(z49627) -> ((exists z49626.((z49626 = _he) & TrueP & (z49627 = z49626)) & TrueP) -> exists e049631.(_on(e049631,z49627) & (Subj(e049631) = z49628) & TrueP))) & exists e049632.((Subj(e049632) = Subj(e049632)) & TrueP)) & exists z49629.(_center(z49629) & TrueP & exists e049